In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from datetime import datetime
import json

In [ ]:
file = "MasterData_Wilson.xlsx"
pd_file = pd.read_excel(file)
bakery_df = pd.DataFrame(pd_file)
bakery_df.head()

In [ ]:
bakery_df.columns

In [ ]:
bakery_df['Unnamed: 29'].value_counts()

In [ ]:
bakery_df.loc[(bakery_df['Unnamed: 29'] == 'Sat') | (bakery_df['Unnamed: 29'] == 2), :]

In [ ]:
bakery_df['Unnamed: 30'].value_counts()

In [ ]:
bakery_df = bakery_df[['date', 'time', 'year', 'MonthN', 'Month', 'WeekN', 'Day', 'WeekD', 'H',
       'Min', '30min', 'Rtime', '30time', 'H.5', 'Operation', 'Line Item',
       'Department', 'Category', 'Supplier', 'Supplier Code', 'Cost', 'Price',
       'Quantity', 'Modifiers', 'Subtotal', 'Discounts', 'Net Total', 'Tax',
       'Total Due']]

In [ ]:
bakery_df.head()

In [ ]:
bakery_df.dtypes

In [ ]:
bakery_df['year'] = bakery_df['year'].fillna(0.0).astype(int)

In [ ]:
bakery_df['year'].value_counts()

In [ ]:
bakery_df.loc[bakery_df['year'] == 0, :]

In [ ]:
bakery_df = bakery_df.drop(index=416580)
bakery_df.loc[bakery_df['year'] == 0, :]

In [ ]:
bakery_df['MonthN'] = bakery_df['MonthN'].fillna(0.0).astype(int)

In [ ]:
bakery_df['MonthN'].value_counts()

In [ ]:
bakery_df['WeekN'] = bakery_df['WeekN'].fillna(0.0).astype(int)
bakery_df['WeekN'].value_counts()

In [ ]:
bakery_df['Day'] = bakery_df['Day'].fillna(0.0).astype(int)
bakery_df['Day'].value_counts()

In [ ]:
bakery_df.head()

In [ ]:
cln_bakery_df = bakery_df[['time', 'WeekN', 'WeekD', 'Rtime', 'Operation', 'Line Item', 'Department', \
                           'Category', 'Supplier', 'Supplier Code', 'Cost', 'Price','Quantity', \
                           'Modifiers', 'Subtotal', 'Discounts', 'Net Total', 'Tax', 'Total Due']]

In [ ]:
cln_bakery_df.head()

In [ ]:
cln_bakery_df['Operation'].value_counts()

In [ ]:
cln_bakery_df['Supplier'].value_counts()

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
damen_reviews = []
belmont_reviews = []
logan_reviews = []

damen_bakeries = ["https://www.yelp.com/biz/la-boulangerie-chicago-4?sort_by=date_desc", \
                  "https://www.yelp.com/biz/la-boulangerie-chicago-4?start=20&sort_by=date_desc", \
                  "https://www.yelp.com/biz/la-boulangerie-chicago-4?start=40&sort_by=date_desc", \
                  "https://www.yelp.com/biz/la-boulangerie-chicago-4?start=60&sort_by=date_desc"]
belmont_bakeries = ['https://www.yelp.com/biz/la-boulangerie-chicago-2?sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=20&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=40&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=60&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=80&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=100&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=120&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=140&sort_by=date_desc', \
                       'https://www.yelp.com/biz/la-boulangerie-chicago-2?start=160&sort_by=date_desc']
logan_bakeries = ['https://www.yelp.com/biz/la-boulangerie-chicago-8?sort_by=date_desc', \
                 'https://www.yelp.com/biz/la-boulangerie-chicago-8?start=20&sort_by=date_desc']


In [4]:
def yelp_bakery_lookup(bakery_list, review_list):
    for link in bakery_list:
        bakery_response = requests.get(link)
        bakery_soup = bs(bakery_response.text, 'html.parser')
        bakery_results = bakery_soup.body.find_all('div', class_='review--with-sidebar')
        for review in bakery_results:
            user_info = []
            user_stats = []
            user_name = '' 
            user_loc = ''
            user_friends = ''
            user_reviews = ''
            user_photos = ''
            review_info = []
            review_stars = ''
            review_date = ''
            review_text = ''
            user_check_ins = ''
            elite = ''
            elite_status = ''
            photo = []
            photo_links = []
            review_data = []
            previous_review = []
            previous_rating = []

            user_information = {}

            try:
                user_info = review.find('ul', class_='user-passport-info')
                user_stats = review.find('ul', class_='user-passport-stats')
                review_info = review.find('div', class_='review-content')
                review_footer = review.find('div', class_='review-footer')

                try:
                    user_name = user_info.find('li', class_='user-name').find('a').text.strip()
                    user_profile_link = user_info.find('a', class_='user-display-name').attrs['href']
                    user_profile_link = "https://www.yelp.com" + user_profile_link
                    user_loc = user_info.find('li', class_='user-location').find('b').text.strip()
                    review_stars = review_info.find('div', class_='i-stars').attrs['title']
                    review_stars = int(float(review_stars.split()[0]))
                    review_date = review_info.find('span', class_='rating-qualifier').text.strip()
                    review_text = review_info.find('p').text
                except AttributeError as error:
                    print(error)

                try:
                    user_friends = user_stats.find('li', class_='friend-count').find('b').text.strip()
                except AttributeError as error:
                    if user_stats:
                        user_friends = '0'

                try:
                    user_reviews = user_stats.find('li', class_='review-count').find('b').text.strip()
                except AttributeError as error:
                    if user_stats:
                        user_reviews = '0'

                try:
                    user_photos = user_stats.find('li', class_='photo-count').find('b').text.strip()
                except AttributeError as error:
                    if user_stats:
                        user_photos = '0'

                try:
                    elite = user_stats.find('li', class_='is-elite')
                    if elite:
                        elite_status = 'elite'
                    else:
                        elite_status = 'not elite'
                except AttributeError as error:
                    print(error)

                try:
                    user_check_ins = review_info.find('li', class_='review-tags_item').find('span').text.strip()
                    user_check_ins = user_check_ins.split()[0]
                except AttributeError as error:
                    if review_info:
                        user_check_ins = '0'

                try:
                    photos = review_info.find('ul', class_='photo-box-grid')
                    if photos:
                        try:
                            picture = photos.find_all('div', class_='photo-box')
                            for pic in picture:
                                pic_link = ''
                                pic_link = pic.find('img').attrs['data-async-src']
                                photo_links.append(pic_link)
                        except Exception as error:
                            print(error)

                except AttributeError as error:
                    if review_info:
                        photo_links = []

                try:
                    review_options = review_footer.find_all('li', class_='vote-item')
                    review_data = {
                        'review_type' : [],
                        'review_count' : []
                    }
                    for option in review_options:
                        review_type = ''
                        review_count = ''
                        review_type = option.find('a').find('span', class_='vote-type').text.strip()
                        review_count = option.find('a').find('span', class_='count').text.strip()
                        if review_count == '':
                            review_count = '0'
                        review_data['review_type'].append(review_type)
                        review_data['review_count'].append(review_count)
                except AttributeError as error:
                    print(error)

                try:
                    previous_review = review.find('div', class_='previous-review')
                    if previous_review:
                        prev_star = previous_review.find('div', class_='i-stars').attrs['title']
                        prev_star = int(float(prev_star.split()[0]))
                        prev_date = previous_review.find('span', class_='rating-qualifier').text.strip()
                        prev_text = previous_review.find('span', class_=['hidden']).text.strip()
                        prev_review_options = previous_review.find_all('li', class_='vote-item')
                        prev_review_data = {
                            'review_type' : [],
                            'review_count' : []
                        }
                        for option in prev_review_options:
                            review_type = ''
                            review_count = ''
                            review_type = option.find('a').find('span', class_='vote-type').text.strip()
                            review_count = option.find('a').find('span', class_='count').text.strip()
                            if review_count == '':
                                review_count = '0'
                            prev_review_data['review_type'].append(review_type)
                            prev_review_data['review_count'].append(review_count)
                        previous_rating = {
                            'stars' : prev_star,
                            'review_date' : prev_date,
                            'review_text' : prev_text,
                            'review_feedback' : prev_review_data
                        }
                except AttributeError as error:
                    previous_review = []
                    print(error)

                if (user_name and user_loc and user_friends and user_reviews and user_photos and review_stars \
                    and review_date and user_check_ins and user_profile_link and review_text):
                    try:
                        user_check_ins = int(user_check_ins)
                    except ValueError as error:
                        user_check_ins = 0
                    counts = review_data['review_count']
                    user_information = {
                        'user_name' : user_name,
                        'user_location' : user_loc,
                        'user_link' : user_profile_link,
                        'user_friends' : int(user_friends),
                        'user_reviews' : int(user_reviews),
                        'user_photos' : int(user_photos),
                        'stars' : review_stars,
                        'review_date' : review_date,
                        'review_text' : review_text,
                        'review_photos' : photo_links,
                        'useful_rating_count' : counts[0],
                        'funny_rating_count' : counts[1],
                        'cool_rating_count' : counts[2],
                        'check_ins' : user_check_ins,
                        'elite_status' : elite_status,
                        'previous_review' : previous_rating
                    }

                    review_list.append(user_information)
            except AttributeError as error:
                print(error)

#     print(json.dumps(review_list, indent=4, sort_keys=True))

In [5]:
yelp_bakery_lookup(damen_bakeries, damen_reviews)
yelp_bakery_lookup(belmont_bakeries, belmont_reviews)
yelp_bakery_lookup(logan_bakeries, logan_reviews)

'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find'
'NoneT

In [6]:
print(len(damen_reviews))
print(len(belmont_reviews))
print(len(logan_reviews))

71
176
37


In [7]:
damen_review_df = pd.DataFrame(damen_reviews).set_index('user_name')
damen_review_df.head()

,check_ins,cool_rating_count,elite_status,funny_rating_count,previous_review,review_date,review_photos,review_text,stars,useful_rating_count,user_friends,user_link,user_location,user_photos,user_reviews
user_name,,,,,,,,,,,,,,,
Sonja M.,0,0,not elite,0,"{'stars': 5, 'review_date': '6/2/2018 ...",1/3/2019\n \n\n\n\n\n\nUpdated review,[],I just stopped by this cafe for the first time...,5,0,63,https://www.yelp.com/user_details?userid=VtVHE...,"Chicago, IL",2,22
Courtney J.,0,1,elite,1,[],12/31/2018,[],I adore this neighborhood bakery. It is a few ...,5,0,263,https://www.yelp.com/user_details?userid=aqkJ_...,"Chicago, IL",10,61
Sabrina G.,1,0,elite,0,[],12/23/2018,[https://s3-media4.fl.yelpcdn.com/bphoto/SqCN8...,I've been dreaming of quiche and croissants as...,5,0,195,https://www.yelp.com/user_details?userid=LlGh8...,"Chicago, IL",82,52
Alfie B.,0,1,elite,0,[],10/10/2018,[https://s3-media4.fl.yelpcdn.com/bphoto/oLYVK...,We came to La Boulangerie after recommendation...,5,0,16,https://www.yelp.com/user_details?userid=mymQs...,"Columbia, SC",323,106
Anna L.,0,1,elite,1,[],10/9/2018,[],This is a great breakfast spot. Even better if...,5,1,28,https://www.yelp.com/user_details?userid=1aah9...,"Columbia, SC",145,176


In [8]:
damen_review_df.columns

Index(['check_ins', 'cool_rating_count', 'elite_status', 'funny_rating_count',
       'previous_review', 'review_date', 'review_photos', 'review_text',
       'stars', 'useful_rating_count', 'user_friends', 'user_link',
       'user_location', 'user_photos', 'user_reviews'],
      dtype='object')

In [9]:
damen_review_df = damen_review_df[['review_date', 'stars', 'user_location', 'review_text', 'check_ins', 'cool_rating_count', 'useful_rating_count', \
                       'funny_rating_count', 'review_photos', 'previous_review', 'elite_status', 'user_link', 'user_friends', \
                       'user_photos', 'user_reviews']]
damen_review_df.head()

,review_date,stars,user_location,review_text,check_ins,cool_rating_count,useful_rating_count,funny_rating_count,review_photos,previous_review,elite_status,user_link,user_friends,user_photos,user_reviews
user_name,,,,,,,,,,,,,,,
Sonja M.,1/3/2019\n \n\n\n\n\n\nUpdated review,5,"Chicago, IL",I just stopped by this cafe for the first time...,0,0,0,0,[],"{'stars': 5, 'review_date': '6/2/2018 ...",not elite,https://www.yelp.com/user_details?userid=VtVHE...,63,2,22
Courtney J.,12/31/2018,5,"Chicago, IL",I adore this neighborhood bakery. It is a few ...,0,1,0,1,[],[],elite,https://www.yelp.com/user_details?userid=aqkJ_...,263,10,61
Sabrina G.,12/23/2018,5,"Chicago, IL",I've been dreaming of quiche and croissants as...,1,0,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/SqCN8...,[],elite,https://www.yelp.com/user_details?userid=LlGh8...,195,82,52
Alfie B.,10/10/2018,5,"Columbia, SC",We came to La Boulangerie after recommendation...,0,1,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/oLYVK...,[],elite,https://www.yelp.com/user_details?userid=mymQs...,16,323,106
Anna L.,10/9/2018,5,"Columbia, SC",This is a great breakfast spot. Even better if...,0,1,1,1,[],[],elite,https://www.yelp.com/user_details?userid=1aah9...,28,145,176


In [10]:
damen_review_df.to_csv('damen_reviews.csv')

In [11]:
belmont_review_df = pd.DataFrame(belmont_reviews).set_index('user_name')
belmont_review_df = belmont_review_df[['review_date', 'stars', 'user_location', 'review_text', 'check_ins', 'cool_rating_count', 'useful_rating_count', \
                       'funny_rating_count', 'review_photos', 'previous_review', 'elite_status', 'user_link', 'user_friends', \
                       'user_photos', 'user_reviews']]
belmont_review_df.head()
belmont_review_df.to_csv('belmont_reviews.csv')


In [12]:
logan_review_df = pd.DataFrame(logan_reviews).set_index('user_name')
logan_review_df = logan_review_df[['review_date', 'stars', 'user_location', 'review_text', 'check_ins', 'cool_rating_count', 'useful_rating_count', \
                       'funny_rating_count', 'review_photos', 'previous_review', 'elite_status', 'user_link', 'user_friends', \
                       'user_photos', 'user_reviews']]
logan_review_df.head()
logan_review_df.to_csv('logan_reviews.csv')


In [13]:
school_link = 'https://www.lyceechicago.org/calendar?mini='
year_list = ['2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02', '2017-03', \
             '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', \
             '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', \
             '2018-10', '2018-11']
event_list=[]

In [14]:
for month in year_list:
    new_link = school_link + month
    school_response = requests.get(new_link)
    school_soup = bs(school_response.text, 'html.parser')
    events = school_soup.find_all('td', class_='has-events')
    for e in events:
        event_link = e.find('div').find('a').attrs['href']
        event_list.append(event_link)

In [15]:
len(event_list)

153

In [ ]:
print(event_list)

In [16]:
event_details = []
for event in event_list:
    date = event.split('/')
    date = date[len(date)-1]
    event_response = requests.get(event)
    event_soup = bs(event_response.text, 'html.parser')
    events_that_day = event_soup.find('div', class_='view-content').find_all('div', class_='views-row')
    for event in events_that_day:
        title = event.find('h3', class_='field-content').find('a').text.strip()
        time_frame = event.find('span', class_='date-display-single').text.strip()
        event_l = event.find('h3', class_='field-content').find('a').attrs['href']
        info = {
            'event_title' : title,
            'event_time' : time_frame,
            'event_date' : date,
            'event_link' : event_l
        }
        event_details.append(info)
print(json.dumps(event_details, indent=4, sort_keys=True))

[
    {
        "event_date": "2016-08-22",
        "event_link": "/calendar/back-to-school-prep-for-staff",
        "event_time": "(All day)",
        "event_title": "Back to School Prep for New Teachers"
    },
    {
        "event_date": "2016-08-23",
        "event_link": "/calendar/back-to-school-prep-for-staff",
        "event_time": "(All day)",
        "event_title": "Back to School Prep for New Teachers"
    },
    {
        "event_date": "2016-08-24",
        "event_link": "/calendar/back-to-school-prep-for-all-staff-0",
        "event_time": "(All day)",
        "event_title": "Back to School Prep for All Staff"
    },
    {
        "event_date": "2016-08-25",
        "event_link": "/calendar/back-to-school-prep-for-all-staff-0",
        "event_time": "(All day)",
        "event_title": "Back to School Prep for All Staff"
    },
    {
        "event_date": "2016-08-26",
        "event_link": "/calendar/back-to-school-prep-for-all-staff-0",
        "event_time": "(All day)",


In [17]:
print(len(event_details))
print(len(event_list))

153
153


In [18]:
for deet in event_details:
    deet['event_link'] = 'https://www.lyceechicago.org' + deet['event_link']
print(event_details)

[{'event_title': 'Back to School Prep for New Teachers', 'event_time': '(All day)', 'event_date': '2016-08-22', 'event_link': 'https://www.lyceechicago.org/calendar/back-to-school-prep-for-staff'}, {'event_title': 'Back to School Prep for New Teachers', 'event_time': '(All day)', 'event_date': '2016-08-23', 'event_link': 'https://www.lyceechicago.org/calendar/back-to-school-prep-for-staff'}, {'event_title': 'Back to School Prep for All Staff', 'event_time': '(All day)', 'event_date': '2016-08-24', 'event_link': 'https://www.lyceechicago.org/calendar/back-to-school-prep-for-all-staff-0'}, {'event_title': 'Back to School Prep for All Staff', 'event_time': '(All day)', 'event_date': '2016-08-25', 'event_link': 'https://www.lyceechicago.org/calendar/back-to-school-prep-for-all-staff-0'}, {'event_title': 'Back to School Prep for All Staff', 'event_time': '(All day)', 'event_date': '2016-08-26', 'event_link': 'https://www.lyceechicago.org/calendar/back-to-school-prep-for-all-staff-0'}, {'eve

In [19]:
lychee_event_df = pd.DataFrame(event_details)
lychee_event_df.head()

,event_date,event_link,event_time,event_title
0,2016-08-22,https://www.lyceechicago.org/calendar/back-to-...,(All day),Back to School Prep for New Teachers
1,2016-08-23,https://www.lyceechicago.org/calendar/back-to-...,(All day),Back to School Prep for New Teachers
2,2016-08-24,https://www.lyceechicago.org/calendar/back-to-...,(All day),Back to School Prep for All Staff
3,2016-08-25,https://www.lyceechicago.org/calendar/back-to-...,(All day),Back to School Prep for All Staff
4,2016-08-26,https://www.lyceechicago.org/calendar/back-to-...,(All day),Back to School Prep for All Staff


In [20]:
lychee_event_df.to_csv('lychee_events.csv')

In [21]:
from pyspark.sql import SparkSession

In [31]:
for index, row in damen_review_df.iterrows():
    if len(row['review_date']) > 15:
        print("Update!")
        damen_review_df.loc[index, 'Updated'] = '1'
        prev = row.previous_review
        damen_review_df.loc[index, 'prev_text'] = prev['review_text']
        damen_review_df.loc[index, 'prev_stars'] = prev['stars']
        damen_review_df.loc[index, 'prev_date'] = prev['review_date']
        counts = prev['review_feedback']['review_count']
        damen_review_df.loc[index, 'prev_useful_rating_count'] = counts[0]
        damen_review_df.loc[index, 'prev_funny_rating_count'] = counts[1]
        damen_review_df.loc[index, 'prev_cool_rating_count'] = counts[2]
    else:
        damen_review_df.loc[index, 'Updated'] = '0'
damen_review_df.head()

Update!
Update!


,review_date,stars,user_location,review_text,check_ins,cool_rating_count,useful_rating_count,funny_rating_count,review_photos,previous_review,...,user_friends,user_photos,user_reviews,Updated,prev_text,prev_stars,prev_date,prev_useful_rating_count,prev_funny_rating_count,prev_cool_rating_count
user_name,,,,,,,,,,,,,,,,,,,,,
Sonja M.,1/3/2019\n \n\n\n\n\n\nUpdated review,5,"Chicago, IL",I just stopped by this cafe for the first time...,0,0,0,0,[],"{'stars': 5, 'review_date': '6/2/2018 ...",...,63,2,22,1,I just stopped by this cafe for the first time...,5.0,6/2/2018\n \nPrevious review,0,0,0
Courtney J.,12/31/2018,5,"Chicago, IL",I adore this neighborhood bakery. It is a few ...,0,1,0,1,[],[],...,263,10,61,0,NaN,NaN,NaN,NaN,NaN,NaN
Sabrina G.,12/23/2018,5,"Chicago, IL",I've been dreaming of quiche and croissants as...,1,0,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/SqCN8...,[],...,195,82,52,0,NaN,NaN,NaN,NaN,NaN,NaN
Alfie B.,10/10/2018,5,"Columbia, SC",We came to La Boulangerie after recommendation...,0,1,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/oLYVK...,[],...,16,323,106,0,NaN,NaN,NaN,NaN,NaN,NaN
Anna L.,10/9/2018,5,"Columbia, SC",This is a great breakfast spot. Even better if...,0,1,1,1,[],[],...,28,145,176,0,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
for index, row in belmont_review_df.iterrows():
    if len(row['review_date']) > 15:
        print("Update!")
        belmont_review_df.loc[index, 'Updated'] = '1'
        prev = row.previous_review
        try:
            belmont_review_df.loc[index, 'prev_text'] = prev['review_text']
            belmont_review_df.loc[index, 'prev_stars'] = prev['stars']
            belmont_review_df.loc[index, 'prev_date'] = prev['review_date']
            counts = prev['review_feedback']['review_count']
            belmont_review_df.loc[index, 'prev_useful_rating_count'] = counts[0]
            belmont_review_df.loc[index, 'prev_funny_rating_count'] = counts[1]
            belmont_review_df.loc[index, 'prev_cool_rating_count'] = counts[2]
        except TypeError as error:
            print(error)
    else:
        belmont_review_df.loc[index, 'Updated'] = '0'
belmont_review_df.head()

Update!
Update!
list indices must be integers or slices, not str
Update!
Update!
Update!
Update!
Update!


,review_date,stars,user_location,review_text,check_ins,cool_rating_count,useful_rating_count,funny_rating_count,review_photos,previous_review,...,user_friends,user_photos,user_reviews,Updated,prev_text,prev_stars,prev_date,prev_useful_rating_count,prev_funny_rating_count,prev_cool_rating_count
user_name,,,,,,,,,,,,,,,,,,,,,
Elise M.,12/27/2018\n \n\n\n\n\n\nUpdated re...,4,"Chicago, IL",2018 Updated Review:This place closed for seve...,0,0,0,0,[],"{'stars': 5, 'review_date': '10/7/2017 ...",...,0,13,36,1,They have a Gourmet Breakfast Sandwich that is...,5.0,10/7/2017\n \nPrevious review,1,0,0
John D.,7/1/2018\n \n\n\n\n\n\nUpdated review,5,"Chicago, IL",Such a beautiful pleasant atmosphere. Highly r...,0,0,0,0,[],[],...,1,0,134,1,NaN,NaN,NaN,NaN,NaN,NaN
Hunny B.,5/3/2018,4,"Chicago, IL",I am reluctant to write this review because I ...,0,0,0,0,[],[],...,5,14,31,0,NaN,NaN,NaN,NaN,NaN,NaN
Tee L.,3/21/2018,4,"Chicago, IL",Cute lil set up with courteous staff. First ti...,0,0,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/-b6RZ...,[],...,7,423,139,0,NaN,NaN,NaN,NaN,NaN,NaN
Rose O.,3/9/2018,4,"Chicago, IL",Their caprese sandwich is just OK but i really...,1,0,0,0,[],[],...,408,1,98,0,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
for index, row in logan_review_df.iterrows():
    if len(row['review_date']) > 15:
        print("Update!")
        logan_review_df.loc[index, 'Updated'] = '1'
        prev = row.previous_review
        try:
            logan_review_df.loc[index, 'prev_text'] = prev['review_text']
            logan_review_df.loc[index, 'prev_stars'] = prev['stars']
            logan_review_df.loc[index, 'prev_date'] = prev['review_date']
            counts = prev['review_feedback']['review_count']
            logan_review_df.loc[index, 'prev_useful_rating_count'] = counts[0]
            logan_review_df.loc[index, 'prev_funny_rating_count'] = counts[1]
            logan_review_df.loc[index, 'prev_cool_rating_count'] = counts[2]
        except TypeError as error:
            print(error)
    else:
        logan_review_df.loc[index, 'Updated'] = '0'
logan_review_df.head()

Update!
Update!


,review_date,stars,user_location,review_text,check_ins,cool_rating_count,useful_rating_count,funny_rating_count,review_photos,previous_review,...,user_friends,user_photos,user_reviews,Updated,prev_text,prev_stars,prev_date,prev_useful_rating_count,prev_funny_rating_count,prev_cool_rating_count
user_name,,,,,,,,,,,,,,,,,,,,,
Erin H.,12/29/2018\n \n\n\n\n\n\nUpdated re...,5,"Chicago, IL",This place has the BEST croissant I've had in ...,1,0,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/efFTu...,"{'stars': 5, 'review_date': '10/8/2018 ...",...,119,15,64,1,This place has the BEST croissant I've had in ...,5.0,10/8/2018\n \nPrevious review,0,0,0
Hope B.,11/13/2018,5,"Chicago, IL",I found this French cafe accidentally when I o...,0,0,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/F-Kg9...,[],...,38,1,3,0,NaN,NaN,NaN,NaN,NaN,NaN
Katherine A.,11/13/2018,5,"Ballwin, MO",Yes. This is what you want. Comfortable feel...,0,0,0,0,[https://s3-media1.fl.yelpcdn.com/bphoto/2vf2F...,[],...,10,145,81,0,NaN,NaN,NaN,NaN,NaN,NaN
Anna G.,11/12/2018,5,"Evanston, IL",Cozy spot for a quick bite or to settle in and...,0,0,1,0,[],[],...,13,89,65,0,NaN,NaN,NaN,NaN,NaN,NaN
Kelly J.,10/19/2018,5,"Chicago, IL",New York has nothing on French bakeries compar...,0,0,0,0,[],[],...,0,0,12,0,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
damen_review_df.columns

Index(['review_date', 'stars', 'user_location', 'review_text', 'check_ins',
       'cool_rating_count', 'useful_rating_count', 'funny_rating_count',
       'review_photos', 'previous_review', 'elite_status', 'user_link',
       'user_friends', 'user_photos', 'user_reviews', 'Updated', 'prev_text',
       'prev_stars', 'prev_date', 'prev_useful_rating_count',
       'prev_funny_rating_count', 'prev_cool_rating_count'],
      dtype='object')

In [37]:
damen_review_df = damen_review_df[['review_date', 'Updated', 'stars', 'user_location', 'review_text', 'check_ins', \
                                   'cool_rating_count', 'useful_rating_count', 'funny_rating_count', 'review_photos',\
                                   'prev_date', 'prev_stars', 'prev_text', 'prev_cool_rating_count', 'prev_useful_rating_count', \
                                   'prev_funny_rating_count', 'elite_status', 'user_link', 'user_friends', 'user_photos', \
                                   'user_reviews']]

In [38]:
belmont_review_df = belmont_review_df[['review_date', 'Updated', 'stars', 'user_location', 'review_text', 'check_ins', \
                                   'cool_rating_count', 'useful_rating_count', 'funny_rating_count', 'review_photos',\
                                   'prev_date', 'prev_stars', 'prev_text', 'prev_cool_rating_count', 'prev_useful_rating_count', \
                                   'prev_funny_rating_count', 'elite_status', 'user_link', 'user_friends', 'user_photos', \
                                   'user_reviews']]

In [39]:
logan_review_df = logan_review_df[['review_date', 'Updated', 'stars', 'user_location', 'review_text', 'check_ins', \
                                   'cool_rating_count', 'useful_rating_count', 'funny_rating_count', 'review_photos',\
                                   'prev_date', 'prev_stars', 'prev_text', 'prev_cool_rating_count', 'prev_useful_rating_count', \
                                   'prev_funny_rating_count', 'elite_status', 'user_link', 'user_friends', 'user_photos', \
                                   'user_reviews']]

In [40]:
damen_review_df.to_csv('damen_reviews.csv')
belmont_review_df.to_csv('belmont_reviews.csv')
logan_review_df.to_csv('logan_reviews.csv')

In [53]:
damen_review_df.loc[damen_review_df['Updated'] == '1', :]

,review_date,Updated,stars,user_location,review_text,check_ins,cool_rating_count,useful_rating_count,funny_rating_count,review_photos,...,prev_stars,prev_text,prev_cool_rating_count,prev_useful_rating_count,prev_funny_rating_count,elite_status,user_link,user_friends,user_photos,user_reviews
user_name,,,,,,,,,,,,,,,,,,,,,
Sonja M.,1/3/2019,1,5,"Chicago, IL",I just stopped by this cafe for the first time...,0,0,0,0,[],...,5.0,I just stopped by this cafe for the first time...,0,0,0,not elite,https://www.yelp.com/user_details?userid=VtVHE...,63,2,22
Allyson M.,4/1/2018,1,4,"Chicago, IL",I dont see that UNfriendly guy anymore in the ...,0,0,0,0,[],...,3.0,I would really like to give this place a 5 bec...,0,0,1,not elite,https://www.yelp.com/user_details?userid=iWYTd...,0,0,12


In [52]:
for index, row in damen_review_df.iterrows():
    if len(row['review_date']) > 15:
        damen_review_df.loc[index, 'review_date'] = row['review_date'].split('\n')[0]
        damen_review_df.loc[index, 'prev_date'] = row['prev_date'].split('\n')[0]

In [55]:
counter = 0
for index, row in belmont_review_df.iterrows():
    if len(row['review_date']) > 15:
        counter += 1
        print(counter)
        try:
            belmont_review_df.loc[index, 'review_date'] = row['review_date'].split('\n')[0]
            belmont_review_df.loc[index, 'prev_date'] = row['prev_date'].split('\n')[0]
        except AtrributeError as error:
            print(error)

1
2
3
4
5


In [56]:
counter = 0
for index, row in logan_review_df.iterrows():
    if len(row['review_date']) > 15:
        counter += 1
        print(counter)
        try:
            logan_review_df.loc[index, 'review_date'] = row['review_date'].split('\n')[0]
            logan_review_df.loc[index, 'prev_date'] = row['prev_date'].split('\n')[0]
        except AtrributeError as error:
            print(error)

1
2


In [60]:
damen_review_df.head()

,Date,Updated,Stars,Location,Text,check_ins,Cool,Useful,Funny,Photos,...,prev_stars,prev_text,prev_Cool,prev_Useful,prev_Funny,status,user_link,user_friends,user_photos,user_reviews
user_name,,,,,,,,,,,,,,,,,,,,,
Sonja M.,1/3/2019,1,5,"Chicago, IL",I just stopped by this cafe for the first time...,0,0,0,0,[],...,5.0,I just stopped by this cafe for the first time...,0,0,0,not elite,https://www.yelp.com/user_details?userid=VtVHE...,63,2,22
Courtney J.,12/31/2018,0,5,"Chicago, IL",I adore this neighborhood bakery. It is a few ...,0,1,0,1,[],...,NaN,NaN,NaN,NaN,NaN,elite,https://www.yelp.com/user_details?userid=aqkJ_...,263,10,61
Sabrina G.,12/23/2018,0,5,"Chicago, IL",I've been dreaming of quiche and croissants as...,1,0,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/SqCN8...,...,NaN,NaN,NaN,NaN,NaN,elite,https://www.yelp.com/user_details?userid=LlGh8...,195,82,52
Alfie B.,10/10/2018,0,5,"Columbia, SC",We came to La Boulangerie after recommendation...,0,1,0,0,[https://s3-media4.fl.yelpcdn.com/bphoto/oLYVK...,...,NaN,NaN,NaN,NaN,NaN,elite,https://www.yelp.com/user_details?userid=mymQs...,16,323,106
Anna L.,10/9/2018,0,5,"Columbia, SC",This is a great breakfast spot. Even better if...,0,1,1,1,[],...,NaN,NaN,NaN,NaN,NaN,elite,https://www.yelp.com/user_details?userid=1aah9...,28,145,176


In [58]:
damen_review_df = damen_review_df.rename(columns={'review_date' : 'Date', 'stars': 'Stars', 'user_location' : \
                                                  'Location', 'review_text' : 'Text', 'check-ins' : 'Check-Ins', \
                                                 'cool_rating_count' : 'Cool', 'useful_rating_count' : 'Useful', \
                                                 'funny_rating_count' : 'Funny', 'review_photos' : 'Photos', \
                                                 'prev_cool_rating_count' : 'prev_Cool', 'prev_useful_rating_count' : \
                                                 'prev_Useful', 'prev_funny_rating_count': 'prev_Funny', 'elite_status' : \
                                                 'status'})

In [61]:
belmont_review_df = belmont_review_df.rename(columns={'review_date' : 'Date', 'stars': 'Stars', 'user_location' : \
                                                  'Location', 'review_text' : 'Text', 'check-ins' : 'Check-Ins', \
                                                 'cool_rating_count' : 'Cool', 'useful_rating_count' : 'Useful', \
                                                 'funny_rating_count' : 'Funny', 'review_photos' : 'Photos', \
                                                 'prev_cool_rating_count' : 'prev_Cool', 'prev_useful_rating_count' : \
                                                 'prev_Useful', 'prev_funny_rating_count': 'prev_Funny', 'elite_status' : \
                                                 'status'})

In [62]:
logan_review_df = logan_review_df.rename(columns={'review_date' : 'Date', 'stars': 'Stars', 'user_location' : \
                                                  'Location', 'review_text' : 'Text', 'check-ins' : 'Check-Ins', \
                                                 'cool_rating_count' : 'Cool', 'useful_rating_count' : 'Useful', \
                                                 'funny_rating_count' : 'Funny', 'review_photos' : 'Photos', \
                                                 'prev_cool_rating_count' : 'prev_Cool', 'prev_useful_rating_count' : \
                                                 'prev_Useful', 'prev_funny_rating_count': 'prev_Funny', 'elite_status' : \
                                                 'status'})

In [74]:
damen_review_df.to_csv('damen_reviews.csv')
belmont_review_df.to_csv('belmont_reviews.csv')
logan_review_df.to_csv('logan_reviews.csv')

In [64]:
damen_review_df.loc[damen_review_df['Updated'] == '1', :]

,Date,Updated,Stars,Location,Text,check_ins,Cool,Useful,Funny,Photos,...,prev_stars,prev_text,prev_Cool,prev_Useful,prev_Funny,status,user_link,user_friends,user_photos,user_reviews
user_name,,,,,,,,,,,,,,,,,,,,,
Sonja M.,1/3/2019,1,5,"Chicago, IL",I just stopped by this cafe for the first time...,0,0,0,0,[],...,5.0,I just stopped by this cafe for the first time...,0,0,0,not elite,https://www.yelp.com/user_details?userid=VtVHE...,63,2,22
Allyson M.,4/1/2018,1,4,"Chicago, IL",I dont see that UNfriendly guy anymore in the ...,0,0,0,0,[],...,3.0,I would really like to give this place a 5 bec...,0,0,1,not elite,https://www.yelp.com/user_details?userid=iWYTd...,0,0,12


In [71]:
#Adding information scraper missed for prev review
belmont_review_df.loc[belmont_review_df.index == 'John D.', ['prev_date']] = '12/31/2016'
belmont_review_df.loc[belmont_review_df.index == 'John D.', ['prev_text']] = 'great breakfast croissant, good value! \
Try it'
belmont_review_df.loc[belmont_review_df.index == 'John D.', ['prev_stars']] = '5'
belmont_review_df.loc[belmont_review_df.index == 'John D.', ['prev_Cool']] = '0'
belmont_review_df.loc[belmont_review_df.index == 'John D.', ['prev_Funny']] = '0'
belmont_review_df.loc[belmont_review_df.index == 'John D.', ['prev_Useful']] = '0'

In [72]:
belmont_review_df.loc[belmont_review_df.index == 'John D.', :]

,Date,Updated,Stars,Location,Text,check_ins,Cool,Useful,Funny,Photos,...,prev_stars,prev_text,prev_Cool,prev_Useful,prev_Funny,status,user_link,user_friends,user_photos,user_reviews
user_name,,,,,,,,,,,,,,,,,,,,,
John D.,7/1/2018,1,5,"Chicago, IL",Such a beautiful pleasant atmosphere. Highly r...,0,0,0,0,[],...,5,"great breakfast croissant, good value! Try it",0,0,0,not elite,https://www.yelp.com/user_details?userid=5AL4m...,1,0,134


In [73]:
belmont_review_df.columns

Index(['Date', 'Updated', 'Stars', 'Location', 'Text', 'check_ins', 'Cool',
       'Useful', 'Funny', 'Photos', 'prev_date', 'prev_stars', 'prev_text',
       'prev_Cool', 'prev_Useful', 'prev_Funny', 'status', 'user_link',
       'user_friends', 'user_photos', 'user_reviews'],
      dtype='object')

In [86]:
for index, row in lychee_event_df.iterrows():
    if row['event_time'] == 'All day':
        lychee_event_df.loc[index, 'start_time'] = '12:00 am'
        lychee_event_df.loc[index, 'end_time'] = '11:59 pm'

In [83]:
for index, row in lychee_event_df.iterrows():
    if row['event_time'] != 'All day':
        time_list = row['event_time'].split(' ')
        if len(time_list) > 2:
            lychee_event_df.loc[index, 'start_time'] = time_list[0] + ' ' + time_list[1]
            lychee_event_df.loc[index, 'end_time'] = time_list[3] + ' ' + time_list[4]
        else:
            lychee_event_df.loc[index, 'start_time'] = time_list[0] + ' ' + time_list[1]

In [87]:
lychee_event_df

,event_date,event_link,event_time,event_title,start_time,end_time
0,2016-08-22,https://www.lyceechicago.org/calendar/back-to-...,All day,Back to School Prep for New Teachers,12:00 am,11:59 pm
1,2016-08-23,https://www.lyceechicago.org/calendar/back-to-...,All day,Back to School Prep for New Teachers,12:00 am,11:59 pm
2,2016-08-24,https://www.lyceechicago.org/calendar/back-to-...,All day,Back to School Prep for All Staff,12:00 am,11:59 pm
3,2016-08-25,https://www.lyceechicago.org/calendar/back-to-...,All day,Back to School Prep for All Staff,12:00 am,11:59 pm
4,2016-08-26,https://www.lyceechicago.org/calendar/back-to-...,All day,Back to School Prep for All Staff,12:00 am,11:59 pm
5,2016-08-29,https://www.lyceechicago.org/calendar/welcome-day,All day,Welcome Day,12:00 am,11:59 pm
6,2016-08-30,https://www.lyceechicago.org/calendar/school-o...,All day,School Opens for 1st - 12th grade,12:00 am,11:59 pm
7,2016-08-31,https://www.lyceechicago.org/calendar/school-o...,8:30 am to 11:30 am,School Opens for Preschool,8:30 am,11:30 am
8,2016-09-01,https://www.lyceechicago.org/calendar/parent-a...,8:15 am to 10:00 am,Parent Association Welcome Coffee,8:15 am,10:00 am
9,2016-09-05,https://www.lyceechicago.org/calendar/labor-da...,All day,Labor Day - School Closed,12:00 am,11:59 pm


In [88]:
lychee_event_df.to_csv('lychee_events.csv')

In [89]:
lychee_event_df['event_title'].value_counts()

Spring Recess                                                                                                 24
Holiday Recess - School Closed                                                                                16
Holiday Recess - School closed                                                                                15
Fall Recess                                                                                                   14
Winter Recess - School Closed                                                                                  8
Winter Recess                                                                                                  8
Thanksgiving Break - School Closed                                                                             8
Fall Recess - School closed                                                                                    5
Start'Up Lycée                                                                                  

In [90]:
for index, row in lychee_event_df.iterrows():
    if row['event_title'] == 'Thanksgiving - School closed':
        lychee_event_df.loc[index, 'event_title'] = 'Thanksgiving Break - School Closed'
    if row['event_title'] == 'Fall Recess':
        lychee_event_df.loc[index, 'event_title'] = 'Fall Recess - School Closed'  
    if row['event_title'] == 'Fall Recess - School closed':
        lychee_event_df.loc[index, 'event_title'] = 'Fall Recess - School Closed' 
    if row['event_title'] == 'Holiday Recess - School closed':
        lychee_event_df.loc[index, 'event_title'] = 'Holiday Recess - School Closed'
    if row['event_title'] == 'Winter Recess':
        lychee_event_df.loc[index, 'event_title'] = 'Winter Recess - School Closed'   

In [91]:
lychee_event_df['event_title'].value_counts()

Holiday Recess - School Closed                                                                                31
Spring Recess                                                                                                 24
Fall Recess - School Closed                                                                                   19
Winter Recess - School Closed                                                                                 16
Thanksgiving Break - School Closed                                                                            11
Start'Up Lycée                                                                                                 4
Back to School Prep for All Staff                                                                              3
Lycée French Market                                                                                            3
Labor Day - School Closed                                                                       

In [92]:
for index, row in lychee_event_df.iterrows():
    if row['event_title'] == 'Memorial Day – School Closed':
        lychee_event_df.loc[index, 'event_title'] = 'Memorial Day - School Closed'
    if row['event_title'] == 'Parent - teacher Conferences - No school for students':
        lychee_event_df.loc[index, 'event_title'] = 'Parent - Teacher Conferences - No School for Students'  
    if row['event_title'] == 'Parent –Teacher Conferences - No School for Students':
        lychee_event_df.loc[index, 'event_title'] = 'Parent - Teacher Conferences - No School for Students' 
    if row['event_title'] == 'In Service Day for Staff - No School for students':
        lychee_event_df.loc[index, 'event_title'] = 'In Service Day for Staff - No School for Students' 

In [93]:
lychee_event_df['event_title'].value_counts()

Holiday Recess - School Closed                                                                                31
Spring Recess                                                                                                 24
Fall Recess - School Closed                                                                                   19
Winter Recess - School Closed                                                                                 16
Thanksgiving Break - School Closed                                                                            11
Start'Up Lycée                                                                                                 4
In Service Day for Staff - No School for Students                                                              4
Labor Day - School Closed                                                                                      3
Lycée French Market                                                                             

In [94]:
lychee_event_df.to_csv('lychee_events.csv')